In [19]:
import os
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import lit
from functools import partial


In [2]:
spark_home = os.environ.get("SPARK_HOME")
# Add Spark bin and executors to PATH
os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "bin")
os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "sbin")

# Add Spark Python libraries to PYTHONPATH
os.environ["PYTHONPATH"] = os.path.join(spark_home, "python") + os.pathsep + os.environ.get("PYTHONPATH", "")
os.environ["PYTHONPATH"] += os.pathsep + os.path.join(spark_home, "python", "lib")

# Add PySpark to the system path
os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "python", "lib", "pyspark.zip")
os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "python", "lib", "py4j-0.10.9-src.zip")

os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark-Script") \
    .getOrCreate()

### Preprocessing the data

In [4]:
dataset = spark.read.csv("dataset.csv", header=True, inferSchema=True)
dataset = dataset.drop("_c83")
dataset.printSchema()
print(dataset.count())
# print(dataset.rdd)

root
 |-- encounter_id: integer (nullable = true)
 |-- patient_id: integer (nullable = true)
 |-- hospital_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- elective_surgery: integer (nullable = true)
 |-- ethnicity: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- height: double (nullable = true)
 |-- icu_admit_source: string (nullable = true)
 |-- icu_id: integer (nullable = true)
 |-- icu_stay_type: string (nullable = true)
 |-- icu_type: string (nullable = true)
 |-- pre_icu_los_days: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- apache_2_diagnosis: integer (nullable = true)
 |-- apache_3j_diagnosis: double (nullable = true)
 |-- apache_post_operative: integer (nullable = true)
 |-- arf_apache: integer (nullable = true)
 |-- gcs_eyes_apache: integer (nullable = true)
 |-- gcs_motor_apache: integer (nullable = true)
 |-- gcs_unable_apache: integer (nullable = true)
 |-- gcs_verbal_apache

In [39]:
'''
Data Cleaning:
1) Removing missing data/ N/A / unexpected values
2) dealing with outliers 
3) data normalization/standardization
4) transform features to categorial if needed
5) visualizing the data using plots(box/scatter/histograms ...etc)
'''
dataset_without_na = dataset.na.drop(how='any')
print(dataset_without_na.count())
 
# ---------------------- For illustration reasons ----------------------#
# col_names = [name for name, dtype in dataset_without_na.dtypes if dtype == "string"]
# for col_name in col_names:
#     dataset_without_na.groupby(col_name).count().show(truncate=False)

'''
 col1 :
    row 1 = a
    row 2 = b
    row 3 = c
    df[row1] = 1
'''

col_names = [name for name, dtype in dataset_without_na.dtypes if dtype == "string"]
for col_name in col_names:
    col_unique_values = dataset_without_na.select(col_name).distinct()
    if (col_unique_values.count() > 2 ) : 
        for col_value in col_unique_values.toLocalIterator():
            dataset_without_na = dataset_without_na.withColumn(col_value[0],(dataset_without_na[col_name]==col_value[0]).cast("int"))
        dataset_without_na = dataset_without_na.drop(col_name) 

          
# dataset_without_na.show()

56935


Py4JJavaError: An error occurred while calling o6582.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java